# Complete Black-Litterman Optimization

| | | | 
|----------------------------------------------------------------------|-----------------------------------------------|---------------------------------------------------------------------------------|
| n=4 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9  | n=4 $returns \sim N(\mu,\sigma)$ Independent  | n=4 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5)  |
| n=8 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9  | n=8 $returns \sim N(\mu,\sigma)$ Independent  | n=8 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5)  |
| n=16 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9 | n=16 $returns \sim N(\mu,\sigma)$ Independent | n=16 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5) |

using the method laid out in 4.2, Black-Litterman portfolio optimization

using https://python-advanced.quantecon.org/black_litterman.html

## Read Packages

In [3]:
import numpy as np
import pandas as pd
import scipy.stats as stat
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
import random
import trace
import sklearn.covariance

In [4]:
import pypfopt as pyp
import warnings
warnings.filterwarnings("ignore")


## Define Functions

Function that Calculated equillibrium values 

This step generates the sample

In [8]:
def bl(n_assets, n_obs, return_vec):
    '''
    This function evaluates the equillibrium returns of a portfolio and generates the sample
    Inputs: 
    n_assets: Number of assets
    n_obs: Number of observations
    return_vec: A matrix of returns of shape n_obs x n_assets (a np.array)
    Returns: 
    weights: optimal weights
    bl_returns: BL returns
    S: BL risk
    '''
    market_prices = pd.Series(np.random.randn(n_assets)) 
    rng = np.random.default_rng()
    cov_struct = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs)
    S = pyp.risk_models.CovarianceShrinkage(cov_struct).ledoit_wolf()
    delta = pyp.black_litterman.market_implied_risk_aversion(market_prices, risk_free_rate=0.5)
    
    market_prior = pd.Series(np.mean(return_vec + 5, axis = 0))
    view = pd.Series(np.median(return_vec + 5, axis = 0))#np.ones(n_assets)*(1/n_assets)+ 5.0)
    
    bl = pyp.BlackLittermanModel(S, pi=market_prior, absolute_views=view)
    bl_return = bl.bl_returns()

    ef = pyp.EfficientFrontier(bl_return, r)
    bl.bl_weights(delta)
    weights = bl.clean_weights()

    S_bl = bl.bl_cov()
    return weights, bl_return, S_bl

In [17]:
### This is a test - do not run
#r = np.random.uniform(0.8, 0.9, (4,4))
#rng = np.random.default_rng()
#return_vec = rng.multivariate_normal(np.zeros(4), cov = r, size = 100)


#market_prices = pd.Series(np.mean(return_vec)) 
#rng = np.random.default_rng()
#cov_struct = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs)
#S = pyp.risk_models.CovarianceShrinkage(cov_struct).ledoit_wolf()
#delta = pyp.black_litterman.market_implied_risk_aversion(market_prices, risk_free_rate=0.5)
    
#market_prior = pd.Series(np.random.randn(n_assets))
#view = pd.Series(np.ones(n_assets)*(1/n_assets))
    
#bl = pyp.BlackLittermanModel(S, pi=market_prior, absolute_views=view)
#bl_return = bl.bl_returns()

#ef = pyp.EfficientFrontier(bl_return, r)
#bl.bl_weights(delta)
#weights = bl.clean_weights()

#S_bl = bl.bl_cov()
#print(cov_struct, S, S_bl)
#np.mean(return_vec, axis = 0)

In [24]:
def one_corr_optimization(n_assets, n_obs, r):
    '''
    First, simulates portfolios then optimizes. 
    This does 100 replications
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = np.zeros((99,n_assets))
    
    for i in range(99):
        n_portfolios = 500
        np.random.seed(i)
        rng = np.random.default_rng()
        return_vec = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs)
        weights, returns, risks = bl(n_assets, n_obs, return_vec) 
        weights = list(weights.items())
        w = [x[1] for x in weights]
        weight_res[i,:] = w
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.diagonal(np.array(risks))
    return weight_res, return_res, risks_res

In [25]:
def one_corr_optimization_zeros(n_assets, n_obs, r):
    '''
    First, simulates portfolios then optimizes. 
    This does 100 replications
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = np.zeros((99,n_assets))
    
    for i in range(99):
        np.random.seed(i)
        rng = np.random.default_rng()
        return_vec = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs)
        _75_perct = int(len(return_vec[:,0])*3/4)
        return_vec[random.sample(list(range(1000)), _75_perct),0] = 0
        np.random.seed(i*5)
        return_vec[random.sample(list(range(1000)), _75_perct),1] = 0
        
        weights, returns, risks = bl(n_assets, n_obs, return_vec)
        weights = list(weights.items())
        w = [x[1] for x in weights]
        weight_res[i,:] = w
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.diagonal(np.array(risks))
    return weight_res, return_res, risks_res

# Perform Optimization
## Correlated

## $n=4$

In [18]:
n_assets = 4
n_obs = 1000
r = np.random.uniform(0.8, 0.9, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [26]:
n4_weights_corr, n4_returns_corr, n4_risks_corr = one_corr_optimization(n_assets, n_obs, r) #

In [27]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_corr, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_corr, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_corr, axis=0),3)))

weights (mean) = [0.25  0.251 0.249 0.25 ] 
returns (mean) = [5.001 5.002 5.001 5.003] 
risks (mean) =[58361975.537 58330435.159 58334748.043 58348001.049]


In [ ]:
print('weights (std) = '+str(np.round(np.std(n4_weights_corr, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_corr, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_corr, axis=0),3)))

In [28]:
n4_weights_corr_os, n4_returns_corr_os, n4_risks_corr_os = one_corr_optimization_zeros(n_assets, n_obs, r)

In [29]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_corr_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_corr_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_corr_os, axis=0),3)))

weights (mean) = [0.251 0.253 0.249 0.246] 
returns (mean) = [5.    5.001 5.004 5.002] 
risks (mean) =[20490931.299 20459769.196 20449191.14  20763641.158]


In [30]:
print('weights (std) = '+str(np.round(np.std(n4_weights_corr_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_corr_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_corr_os, axis=0),3)))

weights (std) = [0.024 0.023 0.024 0.032] 
returns (std) = [0.008 0.008 0.037 0.039] 
risks (std) =[80363448.239 80362921.541 80363032.365 80392527.224]


## $ n = 8$

In [31]:
n_assets = 8
n_obs = 1000
r = r = np.random.uniform(0.8, 0.9, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [32]:
n8_weights_corr, n8_returns_corr, n8_risks_corr = one_corr_optimization(n_assets, n_obs, r) 

In [33]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_corr, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_corr, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_corr, axis=0),3)))

weights (mean) = [0.123 0.124 0.125 0.124 0.126 0.126 0.125 0.126] 
returns (mean) = [4.994 4.994 4.993 4.994 4.995 4.993 4.995 4.993] 
risks (mean) =[71369026.438 71105179.245 71023931.277 71026708.529 71027187.459
 71008524.085 71049610.202 71033945.073]


In [34]:
print('weights (std) = '+str(np.round(np.std(n8_weights_corr, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_corr, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_corr, axis=0),3)))

weights (std) = [0.012 0.012 0.008 0.007 0.005 0.004 0.009 0.005] 
returns (std) = [0.034 0.038 0.034 0.037 0.038 0.037 0.037 0.033] 
risks (std) =[3.35063506e+08 3.35054006e+08 3.35069850e+08 3.35069041e+08
 3.35072102e+08 3.35072748e+08 3.35064570e+08 3.35070701e+08]


In [35]:
n8_weights_corr_os, n8_returns_corr_os, n8_risks_corr_os = one_corr_optimization_zeros(n_assets, n_obs, r)

In [36]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_corr_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_corr_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_corr_os, axis=0),3)))

weights (mean) = [0.125 0.125 0.122 0.126 0.125 0.125 0.126 0.125] 
returns (mean) = [4.999 5.    4.993 4.994 4.993 4.997 4.993 4.993] 
risks (mean) =[19326481.034 19701379.757 19742716.458 19324761.084 20196019.793
 19374487.443 19374018.234 19340012.411]


In [37]:
print('weights (std) = '+str(np.round(np.std(n8_weights_corr_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_corr_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_corr_os, axis=0),3)))

weights (std) = [0.01  0.012 0.019 0.01  0.017 0.013 0.012 0.012] 
returns (std) = [0.008 0.007 0.035 0.035 0.036 0.034 0.034 0.035] 
risks (std) =[49382453.037 49444861.627 49466128.58  49385266.678 49999531.526
 49372956.86  49369375.138 49377595.345]


## $n=16$

In [38]:
n_assets = 16
n_obs = 1000
r = np.random.uniform(0.8, 0.9, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [39]:
n16_weights_corr, n16_returns_corr, n16_risks_corr = one_corr_optimization(n_assets, n_obs, r) 

In [40]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_corr, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_corr, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_corr, axis=0),3)))

weights (mean) = [0.063 0.062 0.063 0.063 0.062 0.062 0.062 0.062 0.063 0.063 0.062 0.063
 0.063 0.063 0.062 0.062] 
returns (mean) = [5.    4.999 4.999 4.998 4.997 4.999 4.997 5.002 5.002 5.002 4.997 5.001
 5.001 5.001 4.999 4.997] 
risks (mean) =[1.88362128e+08 1.88446506e+08 1.88742708e+08 1.88389878e+08
 1.88498229e+08 1.88433493e+08 1.88834628e+08 1.88512220e+08
 1.88363824e+08 1.88355001e+08 1.88398233e+08 1.88370701e+08
 1.88372286e+08 1.88356901e+08 1.88642166e+08 1.89929768e+08]


In [41]:
print('weights (std) = '+str(np.round(np.std(n16_weights_corr, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_corr, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_corr, axis=0),3)))

weights (std) = [0.002 0.005 0.006 0.003 0.004 0.004 0.008 0.005 0.003 0.003 0.006 0.003
 0.003 0.004 0.006 0.008] 
returns (std) = [0.031 0.033 0.028 0.031 0.03  0.031 0.03  0.031 0.029 0.029 0.032 0.03
 0.033 0.033 0.03  0.031] 
risks (std) =[1.26680051e+09 1.26678829e+09 1.26675136e+09 1.26679650e+09
 1.26678167e+09 1.26679037e+09 1.26674065e+09 1.26677997e+09
 1.26680028e+09 1.26680152e+09 1.26679521e+09 1.26679927e+09
 1.26679907e+09 1.26680124e+09 1.26676306e+09 1.26667478e+09]


In [42]:
n16_weights_corr_os, n16_returns_corr_os, n16_risks_corr_os = one_corr_optimization_zeros(n_assets, n_obs, r)

In [43]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_corr_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_corr_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_corr_os, axis=0),3)))

weights (mean) = [0.063 0.062 0.063 0.063 0.062 0.062 0.063 0.063 0.062 0.063 0.062 0.063
 0.063 0.063 0.062 0.062] 
returns (mean) = [5.    5.    5.004 5.003 5.003 5.003 5.001 5.004 5.003 5.001 5.001 5.002
 5.002 5.002 5.    4.999] 
risks (mean) =[1.96301421e+09 1.96318494e+09 1.96301239e+09 1.96304869e+09
 1.96338972e+09 1.96312425e+09 1.96300469e+09 1.96300089e+09
 1.96301904e+09 1.96300166e+09 1.96303687e+09 1.96301096e+09
 1.96301314e+09 1.96300084e+09 1.96304572e+09 1.96311490e+09]


In [44]:
print('weights (std) = '+str(np.round(np.std(n16_weights_corr_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_corr_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_corr_os, axis=0),3)))

weights (std) = [0.003 0.003 0.002 0.002 0.004 0.005 0.002 0.002 0.004 0.001 0.004 0.002
 0.002 0.002 0.005 0.006] 
returns (std) = [0.008 0.007 0.032 0.032 0.033 0.032 0.029 0.033 0.034 0.036 0.035 0.031
 0.032 0.035 0.033 0.032] 
risks (std) =[1.06840197e+10 1.06839889e+10 1.06840200e+10 1.06840135e+10
 1.06839523e+10 1.06839996e+10 1.06840214e+10 1.06840221e+10
 1.06840188e+10 1.06840220e+10 1.06840155e+10 1.06840203e+10
 1.06840199e+10 1.06840221e+10 1.06840139e+10 1.06840013e+10]


## Independent

In [45]:
n_assets = 4
n_obs = 1000
r = np.eye(n_assets)

In [46]:
n4_weights_ind, n4_returns_ind, n4_risks_ind = one_corr_optimization(n_assets, n_obs, r) 

In [47]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_ind, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_ind, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_ind, axis=0),3)))

weights (mean) = [0.252 0.246 0.25  0.252] 
returns (mean) = [5.003 5.    4.997 4.998] 
risks (mean) =[41366410.454 41420949.132 41374162.93  41368818.21 ]


In [48]:
print('weights (std) = '+str(np.round(np.std(n4_weights_ind, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_ind, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_ind, axis=0),3)))

weights (std) = [0.014 0.022 0.017 0.009] 
returns (std) = [0.036 0.03  0.035 0.032] 
risks (std) =[2.25041714e+08 2.25032533e+08 2.25040306e+08 2.25041286e+08]


In [49]:
n4_weights_ind_os, n4_returns_ind_os, n4_risks_ind_os = one_corr_optimization_zeros(n_assets, n_obs, r)

In [50]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_ind_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_ind_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_ind_os, axis=0),3)))

weights (mean) = [0.249 0.252 0.247 0.252] 
returns (mean) = [5.001 5.001 5.005 5.   ] 
risks (mean) =[2.26811071e+08 2.26715110e+08 2.26952088e+08 2.26715803e+08]


In [51]:
print('weights (std) = '+str(np.round(np.std(n4_weights_ind_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_ind_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_ind_os, axis=0),3)))

weights (std) = [0.026 0.019 0.025 0.015] 
returns (std) = [0.007 0.009 0.033 0.035] 
risks (std) =[1.72935528e+09 1.72936727e+09 1.72933855e+09 1.72936717e+09]


## $n=8$

In [52]:
n_assets = 8
n_obs = 1000
r = np.eye(n_assets)

In [53]:
n8_weights_ind, n8_returns_ind, n8_risks_ind = one_corr_optimization(n_assets, n_obs, r) 

In [54]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_ind, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_ind, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_ind, axis=0),3)))

weights (mean) = [0.127 0.124 0.125 0.123 0.126 0.126 0.125 0.124] 
returns (mean) = [4.997 4.997 5.    4.998 4.995 5.002 4.999 5.01 ] 
risks (mean) =[1.41164242e+08 1.41254981e+08 1.41264050e+08 1.41309672e+08
 1.41175698e+08 1.41190328e+08 1.41979280e+08 1.41288954e+08]


In [55]:
print('weights (std) = '+str(np.round(np.std(n8_weights_ind, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_ind, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_ind, axis=0),3)))

weights (std) = [0.006 0.015 0.01  0.014 0.006 0.006 0.013 0.01 ] 
returns (std) = [0.035 0.032 0.032 0.035 0.036 0.031 0.034 0.034] 
risks (std) =[1.15767234e+09 1.15766166e+09 1.15766193e+09 1.15765616e+09
 1.15767101e+09 1.15766935e+09 1.15764631e+09 1.15765807e+09]


In [56]:
n8_weights_ind_os, n8_returns_ind_os, n8_risks_ind_os = one_corr_optimization_zeros(n_assets, n_obs, r)

In [57]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_ind_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_ind_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_ind_os, axis=0),3)))

weights (mean) = [0.124 0.127 0.126 0.127 0.125 0.125 0.122 0.124] 
returns (mean) = [5.    5.    5.    4.996 4.999 5.    4.999 4.999] 
risks (mean) =[1.00710593e+08 1.00290226e+08 1.00321320e+08 1.00288875e+08
 1.00568162e+08 1.00374957e+08 1.02139223e+08 1.00741253e+08]


In [58]:
print('weights (std) = '+str(np.round(np.std(n8_weights_ind_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_ind_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_ind_os, axis=0),3)))

weights (std) = [0.016 0.006 0.007 0.009 0.013 0.012 0.019 0.014] 
returns (std) = [0.008 0.008 0.034 0.036 0.033 0.032 0.034 0.038] 
risks (std) =[4.58954499e+08 4.59027845e+08 4.59020864e+08 4.59027301e+08
 4.58980185e+08 4.59010077e+08 4.58999154e+08 4.58943942e+08]


## $n=16$

In [60]:
n_assets = 16
n_obs = 1000
r = np.eye(n_assets)

In [61]:
n16_weights_ind, n16_returns_ind, n16_risks_ind = one_corr_optimization(n_assets, n_obs, r) 

In [62]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_ind, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_ind, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_ind, axis=0),3)))

weights (mean) = [0.063 0.062 0.062 0.063 0.063 0.062 0.062 0.062 0.063 0.061 0.063 0.063
 0.063 0.062 0.062 0.063] 
returns (mean) = [5.002 4.999 4.994 5.    4.999 4.996 4.998 5.002 4.995 5.    5.005 5.
 5.003 4.997 4.998 5.003] 
risks (mean) =[84472047.856 84485269.367 88592205.958 84431393.951 84455899.423
 84656129.326 84463279.917 84478242.769 84435650.549 84725395.743
 84708431.108 84440187.198 84475077.828 85679986.632 84473093.814
 84439233.516]


In [64]:
print('weights (std) = '+str(np.round(np.std(n16_weights_ind, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_ind, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_ind, axis=0),3)))

weights (std) = [0.003 0.004 0.006 0.004 0.003 0.005 0.004 0.004 0.004 0.008 0.007 0.005
 0.004 0.004 0.006 0.003] 
returns (std) = [0.034 0.038 0.032 0.033 0.033 0.031 0.036 0.031 0.032 0.03  0.036 0.037
 0.032 0.035 0.032 0.034] 
risks (std) =[3.21144975e+08 3.21131210e+08 3.23845349e+08 3.21144501e+08
 3.21138239e+08 3.21148696e+08 3.21137789e+08 3.21132460e+08
 3.21143971e+08 3.21080466e+08 3.21085888e+08 3.21142429e+08
 3.21133494e+08 3.21392719e+08 3.21133735e+08 3.21142265e+08]


In [66]:
n16_weights_ind_os, n16_returns_ind_os, n16_ind_risks_os = one_corr_optimization_zeros(n_assets, n_obs, r)

In [68]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_ind_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_ind_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_ind_risks_os, axis=0),3)))

weights (mean) = [0.061 0.063 0.063 0.063 0.063 0.063 0.063 0.063 0.062 0.063 0.062 0.062
 0.063 0.062 0.062 0.063] 
returns (mean) = [5.001 5.001 4.993 4.998 4.997 4.996 5.    4.998 5.003 5.001 4.998 4.999
 4.997 4.995 5.005 4.999] 
risks (mean) =[1.03423257e+08 1.03227886e+08 1.03208064e+08 1.03226886e+08
 1.03212351e+08 1.03230600e+08 1.03210045e+08 1.03213153e+08
 1.03255032e+08 1.03217907e+08 1.03238442e+08 1.03346370e+08
 1.03214401e+08 1.03221027e+08 1.03344355e+08 1.03210704e+08]


In [69]:
print('weights (std) = '+str(np.round(np.std(n16_weights_ind_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_ind_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_ind_risks_os, axis=0),3)))

weights (std) = [0.008 0.003 0.002 0.002 0.002 0.003 0.002 0.002 0.003 0.002 0.004 0.006
 0.002 0.004 0.006 0.002] 
returns (std) = [0.007 0.008 0.03  0.037 0.034 0.037 0.035 0.036 0.031 0.036 0.033 0.033
 0.036 0.034 0.033 0.032] 
risks (std) =[3.92165310e+08 3.92212204e+08 3.92217260e+08 3.92212640e+08
 3.92216153e+08 3.92211557e+08 3.92216772e+08 3.92215951e+08
 3.92205642e+08 3.92214752e+08 3.92209495e+08 3.92183795e+08
 3.92215626e+08 3.92213928e+08 3.92183642e+08 3.92216588e+08]


## Negatively Correlated

## $n=4$

In [74]:
n_assets = 4
n_obs = 1000
r = r = np.random.uniform(-0.8, -0.5, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [75]:
n4_weights_neg, n4_returns_neg, n4_risks_neg = one_corr_optimization(n_assets, n_obs, r) 

In [76]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_neg, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_neg, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_neg, axis=0),3)))

weights (mean) = [0.252 0.25  0.248 0.249] 
returns (mean) = [4.991 5.003 5.003 4.997] 
risks (mean) =[1.34205046e+08 1.34233612e+08 1.34259989e+08 1.34269039e+08]


In [77]:
print('weights (std) = '+str(np.round(np.std(n4_weights_neg, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_neg, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_neg, axis=0),3)))

weights (std) = [0.013 0.018 0.022 0.022] 
returns (std) = [0.043 0.037 0.04  0.04 ] 
risks (std) =[1.14142861e+09 1.14142532e+09 1.14142239e+09 1.14142160e+09]


In [80]:
n4_weights_neg_os, n4_returns_neg_os, n4_risks_neg_os = one_corr_optimization_zeros(n_assets, n_obs, r)

In [81]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_neg_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_neg_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_neg_os, axis=0),3)))

weights (mean) = [0.245 0.25  0.253 0.252] 
returns (mean) = [4.999 5.001 5.006 5.003] 
risks (mean) =[4.06298611e+08 4.06259933e+08 4.06240020e+08 4.06246835e+08]


In [82]:
print('weights (std) = '+str(np.round(np.std(n4_weights_neg_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_neg_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_neg_os, axis=0),3)))

weights (std) = [0.035 0.027 0.02  0.022] 
returns (std) = [0.011 0.01  0.045 0.042] 
risks (std) =[3.55828886e+09 3.55829325e+09 3.55829551e+09 3.55829474e+09]


## $n=8$

In [85]:
n_assets = 8
n_obs = 1000
r = r = np.random.uniform(-0.8, -0.5, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [86]:
n8_weights_neg, n8_returns_neg, n8_risks_neg = one_corr_optimization(n_assets, n_obs, r) 

In [87]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_neg, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_neg, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_neg, axis=0),3)))

weights (mean) = [0.126 0.127 0.126 0.125 0.124 0.122 0.125 0.126] 
returns (mean) = [4.996 5.002 4.996 5.    4.995 5.007 4.994 5.003] 
risks (mean) =[1.62531183e+08 1.62501807e+08 1.62541940e+08 1.62775557e+08
 1.62712462e+08 1.62882554e+08 1.62531995e+08 1.62517607e+08]


In [88]:
print('weights (std) = '+str(np.round(np.std(n8_weights_neg, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_neg, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_neg, axis=0),3)))

weights (std) = [0.011 0.01  0.01  0.013 0.016 0.018 0.013 0.009] 
returns (std) = [0.048 0.046 0.049 0.046 0.052 0.048 0.044 0.045] 
risks (std) =[1.21819660e+09 1.21820042e+09 1.21819552e+09 1.21816814e+09
 1.21817365e+09 1.21815383e+09 1.21819658e+09 1.21819837e+09]


In [89]:
n8_weights_neg_os, n8_returns_neg_os, n8_risks_neg_os = one_corr_optimization_zeros(n_assets, n_obs, r)

In [90]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_neg_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_neg_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_neg_os, axis=0),3)))

weights (mean) = [0.126 0.125 0.122 0.124 0.125 0.125 0.126 0.127] 
returns (mean) = [5.    5.    4.999 5.003 5.005 4.996 5.002 5.003] 
risks (mean) =[43475363.595 43451051.369 43539793.923 43494707.403 43549804.317
 43463337.628 43493135.641 43454228.897]


In [91]:
print('weights (std) = '+str(np.round(np.std(n8_weights_neg_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_neg_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_neg_os, axis=0),3)))

weights (std) = [0.012 0.009 0.015 0.013 0.013 0.009 0.011 0.009] 
returns (std) = [0.011 0.01  0.05  0.041 0.047 0.052 0.045 0.049] 
risks (std) =[1.57288003e+08 1.57294193e+08 1.57272035e+08 1.57285533e+08
 1.57271436e+08 1.57290992e+08 1.57284416e+08 1.57296817e+08]


## $n=16$

In [92]:
n_assets = 16
n_obs = 1000
r = r = np.random.uniform(-0.8, -0.5, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [93]:
n16_weights_neg, n16_returns_neg, n16_risks_neg = one_corr_optimization(n_assets, n_obs, r) 

In [94]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_neg, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_neg, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_neg, axis=0),3)))

weights (mean) = [0.063 0.063 0.063 0.062 0.063 0.062 0.063 0.062 0.063 0.063 0.061 0.063
 0.062 0.063 0.062 0.063] 
returns (mean) = [5.004 5.004 5.002 4.993 5.001 5.001 4.991 4.998 5.004 4.992 4.992 4.998
 5.003 4.993 5.002 4.993] 
risks (mean) =[89022720.013 88822370.167 88954223.816 88767055.623 88750580.809
 88762651.569 88750199.795 88742435.26  88738380.137 88754234.1
 88869480.71  88737452.771 88762099.267 88734509.496 88752760.405
 88749680.522]


In [95]:
print('weights (std) = '+str(np.round(np.std(n16_weights_neg, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_neg, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_neg, axis=0),3)))

weights (std) = [0.006 0.003 0.006 0.003 0.003 0.005 0.002 0.003 0.002 0.003 0.007 0.003
 0.007 0.002 0.005 0.004] 
returns (std) = [0.056 0.046 0.05  0.043 0.05  0.049 0.047 0.046 0.045 0.055 0.049 0.054
 0.051 0.047 0.052 0.05 ] 
risks (std) =[4.77522527e+08 4.77553159e+08 4.77531856e+08 4.77558289e+08
 4.77561163e+08 4.77559019e+08 4.77561269e+08 4.77562517e+08
 4.77563552e+08 4.77560435e+08 4.77541238e+08 4.77563427e+08
 4.77558907e+08 4.77563960e+08 4.77560730e+08 4.77561226e+08]


In [96]:
n16_weights_neg_os, n16_returns_neg_os, n16_risks_neg_os = one_corr_optimization_zeros(n_assets, n_obs, r)

In [97]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_neg_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_neg_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_neg_os, axis=0),3)))

weights (mean) = [0.063 0.063 0.063 0.062 0.062 0.063 0.063 0.062 0.063 0.063 0.061 0.062
 0.063 0.063 0.063 0.062] 
returns (mean) = [5.002 5.    4.997 5.    5.009 4.993 5.003 5.005 5.001 5.    4.999 5.001
 5.001 5.007 5.005 5.008] 
risks (mean) =[1.21517929e+09 1.21398150e+09 1.21399952e+09 1.21410781e+09
 1.21406513e+09 1.21399314e+09 1.21403115e+09 1.21556084e+09
 1.21424668e+09 1.21427803e+09 1.21427607e+09 1.22005933e+09
 1.21398907e+09 1.21499894e+09 1.21399337e+09 1.21428311e+09]


In [98]:
print('weights (std) = '+str(np.round(np.std(n16_weights_neg_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_neg_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_neg_os, axis=0),3)))

weights (std) = [0.005 0.002 0.003 0.006 0.005 0.002 0.003 0.006 0.006 0.005 0.007 0.007
 0.002 0.005 0.003 0.005] 
returns (std) = [0.012 0.01  0.046 0.044 0.044 0.048 0.05  0.055 0.048 0.051 0.052 0.054
 0.05  0.051 0.053 0.046] 
risks (std) =[1.09402416e+10 1.09403576e+10 1.09403556e+10 1.09403437e+10
 1.09403483e+10 1.09403563e+10 1.09403523e+10 1.09401958e+10
 1.09403286e+10 1.09403256e+10 1.09403255e+10 1.09398427e+10
 1.09403567e+10 1.09402520e+10 1.09403562e+10 1.09403266e+10]
